In [9]:
import re
import os
import json
path="C:\Project\europepmc-corpus-master.tar\IOB"


for (dirpath, dirnames, filenames) in os.walk(path):
    if len(filenames)>1 :
        for e in [os.path.join(dirpath, f) for f in filenames]:
            json_arr=[]
            if e.endswith('.tsv'):
                file_contents = ""
                with open(e, encoding="utf8") as f:
                    file_contents = f.read()
                sentence=file_contents.split("\n\n")
                # print(sentence)
                for content in sentence :
                    # json_arr=[]
                    json_obj={}
                    str = ""
                    drug_type = ""
                    text=''
                    start=''
                    end=''
                    annotations = []
                    for line in content.split("\n"):
                        annotations_obj = {}
                        split=line.split("\t")
                        text += " "+split[0]
                        text = text.strip(" ")
                        # print(text)
                        if len(split) > 1:
                            if 'B' in split[1] or 'I' in split[1]:
                                str += " "+split[0]
                                re_match = re.match(r"(?:(B|I)-)([a-zA-Z]+)", split[1])
                                drug_type = re_match.group(2) if len(re_match.groups()) > 1 else ""
                            elif 'O' in split[1]:
                                annotations_obj["String"] = str.strip()
                                # print(annotations_obj["String"])
                                annotations_obj["type"] = drug_type
                                for word in annotations_obj["String"].splitlines():
                                    for m in re.finditer(re.escape(word),text):
                                        start=(m.start())
                                        end=(m.end())
                                        annotations_obj['start']=start
                                        annotations_obj['end']=end
                                if str:
                                    annotations.append(annotations_obj)
                                str = ""

                    
                    json_obj["corpus"]="europepmc"
                    json_obj["metadata"]=dirpath.split("\\")[-1]
                    json_obj["id"]=e.split("\\")[-1].replace(".tsv", "")
                    json_obj["Text"] =text
                    json_obj["annotation"]=annotations
                    # print(json_obj["Text"])
                    json_arr.append(json_obj)
                    json_filename = e.split("\\")[-1].replace("tsv", "json")  
                    with open(os.path.join(r"C:\\Project\\europepmc-corpus-master.tar\\json_files", json_filename), "w") as f:
                        json.dump(json_arr, f)
        